# EDA

In [1]:
import pandas as pd
import numpy as np
from etl import safe_literal_eval, dtype_checker, movies_df

2384
11796
Memory used: 244.09 MB
Memory used: 244.09 MB
Memory used: 244.18 MB
Memory used: 244.18 MB


[NbConvertApp] Converting notebook etl.ipynb to script
[NbConvertApp] Writing 25688 bytes to etl.py


In [2]:
movies = pd.read_parquet("parquet_data/movies_parquet.parquet")
credits = pd.read_parquet("parquet_data/credits_parquet.parquet")

### Let's recover the fields vote_count and voten average from the CSV movie file.

In [3]:
movies_csv = pd.read_csv("data/movies_dataset.csv")

C:\Users\jdieg\AppData\Local\Temp\ipykernel_21320\3445815435.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_csv = pd.read_csv("data/movies_dataset.csv")


In [4]:
movies_df = movies_df.merge(movies_csv[["id","vote_count", "vote_average"]], on="id", how="left")

### Missing Data

In [5]:
movies_df.isna().sum()

collection_id           40944
budget                      0
id                          0
original_language          11
production_countries        0
release_date                0
revenue                     0
runtime                   246
spoken_languages            0
title                       0
release_year                0
return_on_investment        0
vote_count                  0
vote_average                0
dtype: int64

Having a look at original language field

In [6]:
movies_df[movies_df["original_language"].isna()].head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average
19595,NaN,0.0,283101,None,"[{'iso_3166_1': 'AT', 'name': 'Austria'}, {'is...",2004-10-11,0.0,95.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Shadowing the Third Man,2004,0.0,0.0,0.0
21621,NaN,0.0,103902,None,"[{'iso_3166_1': 'AU', 'name': 'Australia'}]",2007-08-04,0.0,94.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Unfinished Sky,2007,0.0,8.0,6.4
22849,NaN,0.0,359195,None,"[{'iso_3166_1': 'US', 'name': 'United States o...",1960-03-31,0.0,69.0,"[{'iso_639_1': 'en', 'name': 'English'}]",13 Fighting Men,1960,0.0,0.0,0.0
32155,NaN,0.0,147050,None,"[{'iso_3166_1': 'US', 'name': 'United States o...",1929-10-31,0.0,8.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Lambchops,1929,0.0,2.0,6.5
37410,NaN,0.0,257095,None,"[{'iso_3166_1': 'CZ', 'name': 'Czech Republic'}]",1950-01-26,0.0,87.0,"[{'iso_639_1': 'cs', 'name': 'Český'}]",Prince Bayaya,1950,0.0,1.0,5.0


And those rows with null values in field original_language also has missing values on other field sudh as vote_count, vote_average, etc.. having 3 features missing is a significant for our EDa.. so that the more suitable approach is drop them

In [7]:
movies_df.dropna(subset=["original_language"], inplace=True)

### runtime feature missing values

In [8]:
movies_df[movies_df["runtime"].isna()].head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average
634,NaN,0.0,287305,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-21,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Peanuts – Die Bank zahlt alles,1996,0.0,1.0,4.0
635,NaN,0.0,339428,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-03-14,65335.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Happy Weekend,1996,0.0,0.0,0.0
644,NaN,0.0,278978,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-02-29,0.0,NaN,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Und keiner weint mir nach,1996,0.0,0.0,0.0
801,NaN,0.0,282919,de,"[{'iso_3166_1': 'DE', 'name': 'Germany'}]",1996-06-20,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Diebinnen,1996,0.0,1.0,4.0
863,NaN,0.0,253632,fr,"[{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]",1988-10-08,0.0,NaN,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]",Baton Rouge,1988,0.0,0.0,0.0


Some features like revenue, budget, in consecuents roi (shor for return_on_investment), and some vote_count values in zero. Since this is few missing data, the more suitable approach is dropping those rows.

In [9]:
movies_df.dropna(subset=["runtime"], inplace=True)

In [10]:
movies_df.isna().sum()

collection_id           40694
budget                      0
id                          0
original_language           0
production_countries        0
release_date                0
revenue                     0
runtime                     0
spoken_languages            0
title                       0
release_year                0
return_on_investment        0
vote_count                  0
vote_average                0
dtype: int64

### collection_id feature
collection_id features is an exception here since this refers to more information about certain movies that are related to other ones (there's another dataset from etl that can be imported to merge on this id). A more appropriate feature would be adding a new feture that only indicates if it bellongs to a collection.

In [11]:
movies_df["bellongs_to_collection"] = ~movies_df["collection_id"].isnull()

In [12]:
movies_df.isnull().sum()

collection_id             40694
budget                        0
id                            0
original_language             0
production_countries          0
release_date                  0
revenue                       0
runtime                       0
spoken_languages              0
title                         0
release_year                  0
return_on_investment          0
vote_count                    0
vote_average                  0
bellongs_to_collection        0
dtype: int64

By now, we will allow the collection_id feature for the EDA, and may be drop it latter when will be time for training the recomendation model

# Hipothesys

I would like to know how predominant the language english is in the movies. Parhaps thet would be an outlier. And there are two things about languages: the original language wich the movie was produced and the languages availability for each movie. Let's have a look at this proportions

In [13]:
movies_df[["original_language", "spoken_languages"]]

,original_language,spoken_languages
0,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
1,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."
2,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
3,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
4,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
...,...,...
45433,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
45434,tl,"[{'iso_639_1': 'tl', 'name': ''}]"
45435,en,"[{'iso_639_1': 'en', 'name': 'English'}]"
45436,en,"[{'iso_639_1': 'Unknown', 'name': 'Unknown'}]"


In [14]:
movies_df["spoken_languages"]

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45433             [{'iso_639_1': 'en', 'name': 'English'}]
45434                    [{'iso_639_1': 'tl', 'name': ''}]
45435             [{'iso_639_1': 'en', 'name': 'English'}]
45436        [{'iso_639_1': 'Unknown', 'name': 'Unknown'}]
45437             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45181, dtype: object

This is odd, can you see in the visualization of the cell above the language "tl"? this iso 639 1 code does not have a name.. Let's take a look at those rows..

In [15]:
tl_language_mask = movies_df["spoken_languages"].apply(lambda languages: "tl" in (language["iso_639_1"] for language in languages))

In [16]:
movies_df[tl_language_mask].tail()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average,bellongs_to_collection
42284,NaN,0.0,418378,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2016-10-27,0.0,120.0,"[{'iso_639_1': 'tl', 'name': ''}]",Die Beautiful,2016,0.0,2.0,7.5,False
42977,NaN,0.0,392790,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2016-11-09,0.0,110.0,"[{'iso_639_1': 'tl', 'name': ''}]",Ma' Rosa,2016,0.0,2.0,6.0,False
44176,NaN,0.0,74829,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",1983-06-10,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",The Killing of Satan,1983,0.0,1.0,5.0,False
44574,NaN,0.0,302104,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2014-11-09,0.0,110.0,"[{'iso_639_1': 'tl', 'name': ''}]",That Thing Called Tadhana,2014,0.0,4.0,6.5,False
45434,NaN,0.0,111109,tl,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Century of Birthing,2011,0.0,3.0,9.0,False


and it seems that "tl" shows up as a original_language for any Philipine movie... remind this, in the cell above, we are filtering against spoken_language feature.

And after researching at google, it seems "tl" iso 3166 1 code is related to the language Tagalog. A language spoken by a quarter of the population of the Philpines. Click (or tap) on [here](https://en.wikipedia.org/wiki/Tagalog_language) for further information. Let's fill this data.

In [17]:
(movies_df.loc[tl_language_mask, "spoken_languages"]
 .apply(lambda languages: [
     {"iso_639_1": language["iso_639_1"], "name" : "Tagolog" if language["iso_639_1"] == "tl" else language["name"]} 
     for language in languages]))

297      [{'iso_639_1': 'tl', 'name': 'Tagolog'}, {'iso...
772               [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
4940     [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
6690     [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
8518     [{'iso_639_1': 'ja', 'name': '日本語'}, {'iso_639...
8968     [{'iso_639_1': 'cn', 'name': '广州话 / 廣州話'}, {'i...
9551     [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
9720     [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
9924     [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
13649    [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
14222             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
16046    [{'iso_639_1': 'cn', 'name': '广州话 / 廣州話'}, {'i...
16275             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
17043             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
18517             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
18933             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
19332             [{'iso_639_1': 'tl', 'name': 'Tagolog'

In [19]:
movies_df.loc[tl_language_mask,"spoken_languages"] =  (movies_df.loc[tl_language_mask, "spoken_languages"]
                                    .apply(lambda languages: [
                                        {"iso_639_1": language["iso_639_1"], "name" : "Tagolog" if language["iso_639_1"] == "tl" else language["name"]} 
                                        for language in languages]))

In [31]:
# Lets check the result..
movies_df[tl_language_mask].head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average,bellongs_to_collection
297,NaN,0.0,30304,en,"[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1995-05-05,0.0,94.0,"[{'iso_639_1': 'tl', 'name': 'Tagolog'}, {'iso...",Picture Bride,1995,0.000000,5.0,7.4,False
772,NaN,0.0,183955,en,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",1994-09-09,0.0,100.0,"[{'iso_639_1': 'tl', 'name': 'Tagolog'}]",Midnight Dancers,1994,0.000000,1.0,7.0,False
4940,NaN,344.0,50819,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",2001-09-29,4.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",American Adobo,2001,0.011628,3.0,5.3,False
6690,NaN,1000000.0,38135,en,"[{'iso_3166_1': 'Unknown', 'name': 'Unknown'}]",2000-05-18,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",The Debut,2000,0.000000,4.0,5.3,False
8518,NaN,0.0,19742,ja,"[{'iso_3166_1': 'JP', 'name': 'Japan'}]",1956-12-26,0.0,82.0,"[{'iso_639_1': 'ja', 'name': '日本語'}, {'iso_639...",Rodan,1956,0.000000,22.0,5.7,False


In [64]:
# And let's check the final output.. great, it seems we have already impute Tangolog correctly
movies_df.head()

,collection_id,budget,id,original_language,production_countries,release_date,revenue,runtime,spoken_languages,title,release_year,return_on_investment,vote_count,vote_average,bellongs_to_collection
0,10194,30000000.0,862,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,1995,12.451801,5415.0,7.7,True
1,NaN,65000000.0,8844,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,1995,4.043035,2413.0,6.9,False
2,119050,0.0,15602,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,1995,0.000000,92.0,6.5,True
3,NaN,16000000.0,31357,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,1995,5.090760,34.0,6.1,False
4,96871,0.0,11862,en,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,1995,0.000000,173.0,5.7,True


but... this only makes me think that probably the "tl" iso 639 a code may not be the only one.. let's check if there may be other name in blank

In [27]:
movies_df["spoken_languages"].apply(lambda languages: "" in (language["name"] for language in languages)).sum() 

424

Yep, it is.. and let's see what languages does not have name..

The pandas chaining bellow makes this:

Filter all rows whose field spoken_languages avualutes an empty string ("") in the generator of each language object key name.

Then, from the list of languages of each row, give me only the iso 639 1 code whose name is "". That list give me the first element, then give me only unique values

In [63]:
(movies_df.loc[movies_df["spoken_languages"].apply(lambda languages: "" in (language["name"] for language in languages)), "spoken_languages"]
 .apply(lambda languages: [language["iso_639_1"] for language in languages if language["name"] == ""][0])).unique()

array(['gd', 'mn', 'mk', 'kw', 'nv', 'mi', 'yi', 'ne', 'km', 'iu', 'bo',
       'ty', 'si', 'as', 'sh', 'gn', 'lo', 'xh', 'cr', 'ku', 'hy', 'oc',
       'to', 'ce', 'qu', 'am', 'tg', 'tt', 'se', 'ml', 'co', 'dz', 'ht',
       'ln', 'my', 'sa', 'fy', 'tk', 'ny', 'sc', 'gu', 'mr', 'ug', 'ay',
       'st', 'jv', 'br', 'sg', 'lb', 'ab', 'sm', 'ki', 'tn', 'fo', 'sn',
       'bi', 'ig', 'mh'], dtype=object)

In [42]:
movies_df[movies_df["original_language"] == "mr"]["spoken_languages"][29738]

[{'iso_639_1': 'en', 'name': 'English'},
 {'iso_639_1': 'gu', 'name': ''},
 {'iso_639_1': 'hi', 'name': 'हिन्दी'},
 {'iso_639_1': 'mr', 'name': ''}]

In [39]:
movies_df[movies_df["original_language"] == "mk"]["spoken_languages"][2966]

[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'mk', 'name': ''}]

In [46]:
(movies_df["spoken_languages"]
                            .apply(lambda languages: [
                                {"iso_639_1": language["iso_639_1"], "name" : "Gujarati" if language["iso_639_1"] == "gu" else language["name"]} 
                                for language in languages]))

0                 [{'iso_639_1': 'en', 'name': 'English'}]
1        [{'iso_639_1': 'en', 'name': 'English'}, {'iso...
2                 [{'iso_639_1': 'en', 'name': 'English'}]
3                 [{'iso_639_1': 'en', 'name': 'English'}]
4                 [{'iso_639_1': 'en', 'name': 'English'}]
                               ...                        
45433             [{'iso_639_1': 'en', 'name': 'English'}]
45434             [{'iso_639_1': 'tl', 'name': 'Tagolog'}]
45435             [{'iso_639_1': 'en', 'name': 'English'}]
45436        [{'iso_639_1': 'Unknown', 'name': 'Unknown'}]
45437             [{'iso_639_1': 'en', 'name': 'English'}]
Name: spoken_languages, Length: 45181, dtype: object